In [ ]:
import sys, os
import requests
import json
import numpy as np
import time
import pandas as pd
import datetime as dt
from config import api_key_inCites
from config import api_key_WOS
import clarivate.wos_journals.client
from clarivate.wos_journals.client.api import journals_api
from clarivate.wos_journals.client.model.journal_list import JournalList
from pprint import pprint

## Pip install the Clarivate Journal Client
## pip install git+https://github.com/clarivate/wosjournals-python-client.git

In [ ]:
## Print files in current dirctory
for i in os.listdir('./'):
    print(i)

In [ ]:
## Use this to SQL query in Elements Reporting Database for faculty publication data
## https://github.com/kglibrarian/symplecticelementssql/blob/master/Data%20Source%20Publication%20IDs%20Report%20by%20Group%20of%20Authors.md

## Upload the resulting .csv from Elements

elements_data_path = "data/2023_01-12_FSM_Pubs_2018_to_2022.csv"

## Read the CSV file and store into Pandas DataFrame 
elements_data_df = pd.read_csv(elements_data_path, encoding = "ISO-8859-1")
## encoding = "ISO-8859-1", na_values=['NULL', '<NA>']

#Change the column names to lower case with underscore for spaces
elements_data_df.columns =  elements_data_df.columns.str.strip().str.lower().str.replace(" ", "_").str.replace("(","").str.replace(")","").str.replace("ï»¿","")
elements_data_df.head()

In [ ]:
## Check the length of the dataframe (i.e. how many rows)
print("Number of rows in the dataframe: ", len(elements_data_df.index))

## Check how many nans are in each column
elements_data_df.isnull().sum(axis = 0)

In [ ]:
## Make a column based on year
## 1. Change publication_date column from object to a datetime format
elements_data_df['publication_date'] = pd.to_datetime(elements_data_df['publication_date'], format='%m/%d/%Y')

## 2. Extract year and create new column from publication_date
elements_data_df['year'] = elements_data_df['publication_date'].dt.year 
elements_data_df.head()

In [ ]:
## Summarize how many faculty have published in a journal by year
## Make a new dataframe with subset of data, including user_id, publication_id, issn, eissn, and year
## Our backbone ID is the eissn, and then also year
## a) If publication ID is listed more than once for each journal in each year, we only want to count it once. 
## b) If user ID is listed more than once for each journal in each year, we only want to count it once.

unique_publications_df = elements_data_df.groupby(['year', 'eissn']).agg({'publication_id': 'nunique', 
                                                                          'user_id':'nunique' }).rename(columns={'publication_id': 'Count of Unique Publications',
                                                                                                                'user_id': 'Count of Unique Authors'})
unique_publications_df.head(100)


In [ ]:
## Reset the index so year and eissn are columns
unique_publications_df.reset_index(level=['year','eissn'], inplace=True)
unique_publications_df.head(100)



In [ ]:
## Summarize across 5 years how many unique faculty have published in a journal

unique_publications_all_df = elements_data_df.groupby(['eissn']).agg({'publication_id': 'nunique', 
                                                                          'user_id':'nunique' }).rename(columns={'publication_id': 'Count of Unique Publications 2018-2022',
                                                                                                                'user_id': 'Count of Unique Authors 2018-2022'})
unique_publications_all_df.head(100)

In [ ]:
## Pivot so that the data is provided by Year

pivot_unique_publications_df = unique_publications_df.pivot(index='eissn', columns='year', values=['Count of Unique Publications', 'Count of Unique Authors'])
pivot_unique_publications_df.head(100)

In [ ]:
## Flatten the column names 
pivot_unique_publications_df.columns = [f"{x}_{y}" for x, y in pivot_unique_publications_df.columns.to_flat_index()]
pivot_unique_publications_df.head(100)

In [ ]:
## Identify the column names
# iterating the columns
for col in pivot_unique_publications_df.columns:
    print(col)

In [ ]:
## Sum publication columns for 5 year totals

# pivot_unique_publications_df['Count of Unique Publications 2018-2022'] = pivot_unique_publications_df.loc[:, "Count of Unique Publications_2018":"Count of Unique Publications_2022"].sum(axis=1)
# pivot_unique_publications_df.head(50)

# df['variance'] = df.loc[:,['budget','actual']].sum(axis=1)

In [ ]:
## Upload the .csv from CMMS of Top ILL & Turnaways

cmms_data_path = "data/2023_Top ILL & Turnaways.csv"

## Read the CSV file and store into Pandas DataFrame 
cmms_data_df = pd.read_csv(cmms_data_path, encoding = "ISO-8859-1")
## encoding = "ISO-8859-1", na_values=['NULL', '<NA>']

#Change the column names to lower case with underscore for spaces
cmms_data_df.columns =  cmms_data_df.columns.str.strip().str.lower().str.replace(" ", "_").str.replace("(","").str.replace(")","").str.replace("ï»¿","")
cmms_data_df.head()

In [ ]:
## Check the length of the dataframe (i.e. how many rows)
print("Number of rows in the dataframe: ", len(cmms_data_df.index))

In [ ]:
merged_df = pd.merge(cmms_data_df, pivot_unique_publications_df, left_on='electronic_issn', right_index=True, how="left")
merged_df.head()

In [ ]:
## Check the length of the dataframe (i.e. how many rows)
print("Number of rows in the dataframe: ", len(merged_df.index))

In [ ]:
## Merge with yearly totals

merged_2_df = pd.merge(merged_df, unique_publications_all_df, left_on='electronic_issn', right_index=True, how="left")
merged_2_df.head()

In [ ]:
## Check the length of the dataframe (i.e. how many rows)
print("Number of rows in the dataframe: ", len(merged_2_df.index))

In [ ]:
## Save dataframe to a CSV

with open(r"output/merged_2_df.csv", 'w', encoding='utf-8') as file:
    merged_2_df.to_csv(file, line_terminator='\n', index=True)
    file.close()